In [ ]:
import numpy as np
import random 
import time

In [ ]:
def output_dimacs():
    pass

def decode(state):
    coords = [str(i) + str(j) for i in range(1,10) for j in range(1,10)]
    new_state = []
    for x in range(len(state) - 1):
        if state[x] == '.':
            continue
        else:
            new_state.append([int(coords[x] + state[x])])
    return new_state

def propagate(f, unit_clause):
    modified = []
    for clause in f:
        if unit_clause in clause:
            continue
        if -unit_clause in clause:
            new_clause = [x for x in clause if x != -unit_clause]
            if not new_clause:
                return -1
            modified.append(new_clause)
        else:
            modified.append(clause)
    return modified

def unit_propagation(f):
    assignment = []
    unit_clauses = []  
    unit_clauses = [clause for clause in f if len(clause) == 1]
    while len(unit_clauses) > 0:
        unit_clause = unit_clauses[0]
        f = propagate(f, unit_clause[0])
        assignment += [[unit_clause[0]]]
        if f == -1:
            return -1, []
        if not f:
            return f, assignment
        unit_clauses = [clause for clause in f if len(clause) == 1]
    return f, assignment

def next_literal(f, assignment, heuristic='random'):
    if heuristic == 'random':
        return random.choice(np.setdiff1d(literals, [[abs(x[0])] for x in assignment]))
    #if heurist == 'jw':
        # return literal
    
def backtracking(formula, assignment, literals):
    formula, unit_assignment = unit_propagation(formula)
    assignment = assignment + unit_assignment 
    if formula == - 1:
        return []
    if not formula:
        return assignment
    
    variable = next_literal(formula, assignment, 'random')
    solution = backtracking(propagate(formula, variable), assignment + [[variable]], literals)
    if not solution:
        solution = backtracking(propagate(formula, -variable), assignment + [[-variable]], literals)

    return solution

In [ ]:
# Load in all clauses
with open('sudoku-rules.txt', 'r') as f:
    clauses = [lines.split()[:-1] for lines in f][1:]
    clauses = [ list(np.array(clause, dtype=int)) for clause in clauses]

literals = [[x+y+z] for x in range(100,1000, 100) for y in range(10,100,10) for z in range(1,10)]

with open('top91.sdk.txt', 'r') as f:
    for lines in f:
        start = time.time()
        solution = backtracking(clauses + decode(lines), [], literals)
        end = time.time()
        print('seconds',end - start)

In [ ]:
# flatten solution list so we can easily display it
flatten = lambda *n: (e for a in n
    for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))
solution = list(flatten(solution))

def visualize_solution(state):
    sudoku_grid = np.zeros((9,9))
    for x in state:
        if x > 0:
            sudoku_grid[int(str(x)[0])-1,int(str(x)[1])-1] = int(str(x)[2]) 
    return sudoku_grid

visualize_solution(solution)